In [2]:
import logging

import gym
import torch

import neat.population as pop
import neat.experiments.pole_balancing.config as c
from neat.visualize import draw_net
from neat.phenotype.feed_forward import FeedForwardNet

import time
import optuna

/Users/aidan.horn/opt/anaconda3/envs/EvolvingNeuralNetworks/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-12-22 15:32:23,151 [MainThread  ] [_tools    ] [DEBUG]  deprecate positional args: graphviz.backend.piping.pipe(['renderer', 'formatter', 'neato_no_op', 'quiet'])
2022-12-22 15:32:23,152 [MainThread  ] [_tools    ] [DEBUG]  deprecate positional args: graphviz.backend.rendering.render(['renderer', 'formatter', 'neato_no_op', 'quiet'])
2022-12-22 15:32:23,153 [MainThread  ] [_tools    ] [DEBUG]  deprecate positional args: graphviz.backend.unflattening.unflatten(['stagger', 'fanout', 'chain', 'encoding'])
2022-12-22 15:32:23,154 [MainThread  ] [_tools    ] [DEBUG]  deprecate positional args: graphviz.backend.viewing.view(['quiet'])
2022-12-22 15:32:23,157 [MainThread  ] [_tools    ] [DEBUG]  depr

In [3]:
config = c.PoleBalanceConfig

def objective(trial):

    config.SCALE_ACTIVATION = trial.suggest_float('S_ACT', 1 , 10, log = True)

    config.FITNESS_THRESHOLD = 100000.0

    config.POPULATION_SIZE = trial.suggest_int('POP_SIZE', 5, 200)
    config.NUMBER_OF_GENERATIONS = 100
    config.SPECIATION_THRESHOLD = trial.suggest_float('SPEC_THRESH', 1.0, 5.0)

    config.CONNECTION_MUTATION_RATE = trial.suggest_float('CONN_MUT_RATE', 0.1, 0.9)
    config.CONNECTION_PERTURBATION_RATE = trial.suggest_float('CONN_PERT_RATE', 0.1, 0.9)
    config.ADD_NODE_MUTATION_RATE = trial.suggest_float('ADD_NODE_RATE', 0.1, 0.9)
    config.ADD_CONNECTION_MUTATION_RATE = trial.suggest_float('ADD_CONN_RATE', 0.1, 0.9)

    config.CROSSOVER_REENABLE_CONNECTION_GENE_RATE = trial.suggest_float('CROSSOVER', 0.1, 0.9)

    # Top percentage of species to be saved before mating
    config.PERCENTAGE_TO_SAVE = 0.80

    neat = pop.Population(config)
    start = time.time()
    solution, generation = neat.run()
    end = time.time()

    inv_ttc = -1*(end - start)

    return (inv_ttc)



In [4]:
study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials=1000)

trial = study.best_trial

print('Time to Converge: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))



[I 2022-12-22 15:32:45,118] A new study created in memory with name: no-name-882a7f41-a5f1-431a-ad6d-be9e8b6963e5
2022-12-22 15:32:46,127 [MainThread  ] [population] [INFO ]  Finished Generation 1
2022-12-22 15:32:46,128 [MainThread  ] [population] [INFO ]  Best Genome Fitness: 190.0
2022-12-22 15:32:46,128 [MainThread  ] [population] [INFO ]  Best Genome Length 5

2022-12-22 15:32:46,830 [MainThread  ] [population] [INFO ]  Finished Generation 2
2022-12-22 15:32:46,830 [MainThread  ] [population] [INFO ]  Best Genome Fitness: 285.0
2022-12-22 15:32:46,831 [MainThread  ] [population] [INFO ]  Best Genome Length 5

2022-12-22 15:32:47,638 [MainThread  ] [population] [INFO ]  Finished Generation 3
2022-12-22 15:32:47,639 [MainThread  ] [population] [INFO ]  Best Genome Fitness: 331.0
2022-12-22 15:32:47,639 [MainThread  ] [population] [INFO ]  Best Genome Length 5

2022-12-22 15:32:48,418 [MainThread  ] [population] [INFO ]  Finished Generation 4
2022-12-22 15:32:48,418 [MainThread  ] [p

KeyboardInterrupt: 

In [15]:
optuna.visualization.plot_optimization_history(study)
optuna.visualization.plot_slice(study) 
optuna.visualization.plot_contour(study, params=['S_ACT', 'POP_SIZE', 'SPEC_THRESH', 'CONN_MUT_RATE'], target_name='Time to Converge')

/Users/aidan.horn/opt/anaconda3/envs/EvolvingNeuralNetworks/lib/python3.8/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [18]:
optuna.visualization.plot_contour(study, params=['POP_SIZE', 'CONN_MUT_RATE'], target_name='Time to Converge')

In [19]:
optuna.visualization.plot_contour(study, params=['POP_SIZE', 'SPEC_THRESH'], target_name='Time to Converge')